In [1]:
import pandas as pd
import json
from os import listdir
from os.path import isfile, join

In [2]:
def get_generalData(data):
    '''
    this function take as entry :
        - data : a json file data (corresponding to one match)
    it returns the general metadata of the match:
        - [id_game,season,dateTime,endDateTime,abstractGameState,team_away_name,team_home_name]
    '''
    id_game = data['gameData']['game']['pk']
#     season = data['gameData']['game']['season']
    yearBegin = str(id_game)[:4]
    season = yearBegin + str(int(yearBegin) + 1)
    
    try :
        dateTime =  data['gameData']['datetime']['dateTime']
    except :
        print(f"dateTime not found for match {id_game}")
        dateTime = None
    try :
        endDateTime = data['gameData']['datetime']['endDateTime']
    except :
        #print(f"endDateTime not found for match {id_game}")
        endDateTime = None
    
    abstractGameState = data['gameData']['status']['abstractGameState']
    team_away_name = data['gameData']['teams']['away']['name']
    team_home_name = data['gameData']['teams']['home']['name']
    return [id_game,season,dateTime,endDateTime,abstractGameState,team_away_name,team_home_name]

In [66]:
import datetime
from math import sqrt
import numpy as np
def get_file_event_rows_data(data,type_season):
    ''' 
    this function take as entry :
        - data : a json file data (corresponding to one match of season defined by type_season)
        - type_season : the season in format 20XX20XX (for instance 20162017)
    It ouputs the a list of list, in which each row contains metadata of one event of the match (only shots and goals are considered)
    the metadata the following :
        -  columns_name = [
            "event_Idx","period", "periodTime","id_team_that_shot","name_team_that_shot","result_event","x_coord","y_coord",\
                "rinkSide_of_the_team_that_shot","goalie_name",\
                "shooter_name","shot_type","empty_net","strength",\
                "type_season","id_game","season","dateTime","endDateTime","abstractGameState","team_away_name","team_home_name"]
    '''
    match_data = get_generalData(data)
    team_away_name,team_home_name = match_data[-2],match_data[-1]
    match_events_list = []
    previous_event = None
    for item in data['liveData']['plays']['allPlays']:
        if item['result']['event'] not in ["Goal", "Shot"]:
            previous_event = item
            continue
        else :
            # event Idx
            event_Idx = item['about']['eventIdx']
           
          
            # period
            period = item['about']['period']

            # periodTime
            periodTime = item['about']['periodTime']
            
            
            
           
            # team information (which team shot)
            id_team_that_shot = item['team']['id'] 
            name_team_that_shot = item['team']['name'] 

            # indicator if its a shot or a goal
            result_event = item['result']['event']
            
            # the rinkside of the the_team_that_shot
            
            try : 
                # sometimes we don't have the info on the rinksidee apparently
                if team_away_name == name_team_that_shot :
                    if int(period)%2==0:
                        # in case of prolongation even (knowing that the max prolongation in the history is 6)
                        rinkSide_of_the_team_that_shot = data['liveData']['linescore']['periods'][1]['away']['rinkSide']
                    else :
                         # in case of prolongation odd
                        rinkSide_of_the_team_that_shot = data['liveData']['linescore']['periods'][0]['away']['rinkSide']
                else :
                    if int(period)%2==0:
                        rinkSide_of_the_team_that_shot = data['liveData']['linescore']['periods'][1]['home']['rinkSide']
                    else : 
                        rinkSide_of_the_team_that_shot = data['liveData']['linescore']['periods'][0]['home']['rinkSide']
            except Exception as e :
                #print(e)
                #print(f"period not defined for match {match_data[-7]} and event {event_Idx} and period:{period}")
                rinkSide_of_the_team_that_shot = None
            
            # the on ice coordinates
            try :
                x_coord = item['coordinates']['x'] 
                y_coord = item['coordinates']['y']
                if rinkSide_of_the_team_that_shot == 'left':
                    distance_from_net =  sqrt((90 - x_coord)**2 + (y_coord-0)**2)
                else:
                    distance_from_net =  sqrt((-90 - x_coord)**2 + (y_coord-0)**2)
                
                angle  = np.arcsin(y_coord/distance_from_net)
            except :
                #print(f"coordinates not found for match {match_data[-7]} and event {event_Idx}")
                x_coord = None
                y_coord = None
                distance_from_net = None
                angle = None
            

            # the shooter and goalie name
            goalie_name = None
            shooter_name = None
            for item_bis in item['players']:
                if item_bis['playerType']=="Goalie":
                    goalie_name = item_bis['player']["fullName"]
                elif item_bis['playerType'] in ["Shooter", "Scorer"]:
                    shooter_name = item_bis['player']["fullName"]
                else:
                    continue
            '''
            if goalie_name == None :
                print(f"goalie_name not found for match {match_data[-7]} and event {event_Idx}")
            if shooter_name == None :
                print(f"shooter_name  not found for match {match_data[-7]} and event {event_Idx}")
            '''
            # shot type
            try :
                shot_type = item['result']['secondaryType']
            except :
                # sometimes the secondary Type is not defined
                #print(f"shot_type not found for match {match_data[-7]} and event {event_Idx}")
                shot_type = None

            try : 
                # empty net
                empty_net = item['result']['emptyNet']
            except :
                empty_net = None
            
            # strength
            try :
                strength = item['result']['strength']['name']
            except :
                strength = None
                
            ### PREVIOUS EVENT ###
            # Default values
            rebond = False
            last_x_coord = None
            last_y_coord = None
            distance_from_last_event = None
            speed = None
            change_in_angle = 0
            time_since_last_event = None
            last_event_type = None
            
            if period != previous_event['about']['period']:
                previous_event = None
                
            if previous_event :
                
                # Last event type
                last_event_type = previous_event['result']['event']
                
                # Is rebound
                if last_event_type in ["Goal", "Shot"]:
                    rebond = True

                # time since last event
                format = "%M:%S"
                dt_time = datetime.datetime.strptime(periodTime, format)
                dt_last_time = datetime.datetime.strptime(previous_event['about']['periodTime'], format)
                time_since_last_event = (dt_time - dt_last_time).total_seconds()  
                if time_since_last_event == 0:
                    time_since_last_event = 0.5
                    
               
                try :
                    # the on ice coordinates
                    last_x_coord = previous_event['coordinates']['x'] 
                    last_y_coord = previous_event['coordinates']['y']
                    
                    # distance
                    distance_from_last_event = sqrt((last_y_coord - y_coord)**2 + (last_x_coord - x_coord)**2)
                   
                    # change in angle
                    if rebond:
                        if rinkSide_of_the_team_that_shot == 'left':
                            last_distance_from_net =  sqrt((90 - last_x_coord)**2 + (last_y_coord-0)**2)
                        else:
                            last_distance_from_net =  sqrt((-90 - last_x_coord)**2 + (last_y_coord-0)**2)

                        last_angle  = np.arcsin(last_y_coord/last_distance_from_net)
                        change_in_angle = angle - last_angle
                    else:
                        change_in_angle = 0
                    
                    # speed
                    speed = distance_from_last_event / time_since_last_event        
                except:
                    pass
                
            
            all_data = [event_Idx, period, periodTime, id_team_that_shot,name_team_that_shot,result_event,x_coord,y_coord,distance_from_net,angle,rinkSide_of_the_team_that_shot,goalie_name,shooter_name,shot_type,empty_net,strength,last_event_type,last_x_coord,last_y_coord,distance_from_last_event,time_since_last_event,rebond,change_in_angle,speed]
            match_events_list.append(all_data+[type_season]+match_data)
            previous_event = item
    return match_events_list


In [15]:
# list all the json files extracted in datasets
dir_year = [ join("../datasets/raw/", d_y) for d_y in listdir("../datasets/raw/")]
dir_pl_reg = [(join(path, d_pl_reg),d_pl_reg)  for path in dir_year for d_pl_reg in listdir(path)  ]
fichiers = [(join(dir_path, f),d_pl_reg) \
    for (dir_path, d_pl_reg) in dir_pl_reg
    for f in listdir(dir_path) if isfile(join(dir_path, f))]

In [67]:
# extract all metedata 
all_list_data = []
for (file_name_path, type_season) in fichiers:
    print(file_name_path)
    with open(file_name_path,'r') as f:
        data = json.loads(f.read())
        
        if 'messageNumber' in data and data['messageNumber'] == 2:
            continue
            
        all_list_data = all_list_data + get_file_event_rows_data(data,type_season)

../datasets/raw/2015\playoffs\2015030111.json
../datasets/raw/2015\playoffs\2015030112.json
../datasets/raw/2015\playoffs\2015030113.json
../datasets/raw/2015\playoffs\2015030114.json
../datasets/raw/2015\playoffs\2015030115.json
../datasets/raw/2015\playoffs\2015030116.json
../datasets/raw/2015\playoffs\2015030117.json
../datasets/raw/2015\playoffs\2015030121.json
../datasets/raw/2015\playoffs\2015030122.json
../datasets/raw/2015\playoffs\2015030123.json
../datasets/raw/2015\playoffs\2015030124.json
../datasets/raw/2015\playoffs\2015030125.json
../datasets/raw/2015\playoffs\2015030126.json
../datasets/raw/2015\playoffs\2015030127.json
../datasets/raw/2015\playoffs\2015030131.json
../datasets/raw/2015\playoffs\2015030132.json
../datasets/raw/2015\playoffs\2015030133.json
../datasets/raw/2015\playoffs\2015030134.json
../datasets/raw/2015\playoffs\2015030135.json
../datasets/raw/2015\playoffs\2015030136.json
../datasets/raw/2015\playoffs\2015030137.json
../datasets/raw/2015\playoffs\2015

../datasets/raw/2015\regular_season\2015020074.json
../datasets/raw/2015\regular_season\2015020075.json
../datasets/raw/2015\regular_season\2015020076.json
../datasets/raw/2015\regular_season\2015020077.json
../datasets/raw/2015\regular_season\2015020078.json
../datasets/raw/2015\regular_season\2015020079.json
../datasets/raw/2015\regular_season\2015020080.json
../datasets/raw/2015\regular_season\2015020081.json
../datasets/raw/2015\regular_season\2015020082.json
../datasets/raw/2015\regular_season\2015020083.json
../datasets/raw/2015\regular_season\2015020084.json
../datasets/raw/2015\regular_season\2015020085.json
../datasets/raw/2015\regular_season\2015020086.json
../datasets/raw/2015\regular_season\2015020087.json
../datasets/raw/2015\regular_season\2015020088.json
../datasets/raw/2015\regular_season\2015020089.json
../datasets/raw/2015\regular_season\2015020090.json
../datasets/raw/2015\regular_season\2015020091.json
../datasets/raw/2015\regular_season\2015020092.json
../datasets/

../datasets/raw/2015\regular_season\2015020235.json
../datasets/raw/2015\regular_season\2015020236.json
../datasets/raw/2015\regular_season\2015020237.json
../datasets/raw/2015\regular_season\2015020238.json
../datasets/raw/2015\regular_season\2015020239.json
../datasets/raw/2015\regular_season\2015020240.json
../datasets/raw/2015\regular_season\2015020241.json
../datasets/raw/2015\regular_season\2015020242.json
../datasets/raw/2015\regular_season\2015020243.json
../datasets/raw/2015\regular_season\2015020244.json
../datasets/raw/2015\regular_season\2015020245.json
../datasets/raw/2015\regular_season\2015020246.json
../datasets/raw/2015\regular_season\2015020247.json
../datasets/raw/2015\regular_season\2015020248.json
../datasets/raw/2015\regular_season\2015020249.json
../datasets/raw/2015\regular_season\2015020250.json
../datasets/raw/2015\regular_season\2015020251.json
../datasets/raw/2015\regular_season\2015020252.json
../datasets/raw/2015\regular_season\2015020253.json
../datasets/

../datasets/raw/2015\regular_season\2015020410.json
../datasets/raw/2015\regular_season\2015020411.json
../datasets/raw/2015\regular_season\2015020412.json
../datasets/raw/2015\regular_season\2015020413.json
../datasets/raw/2015\regular_season\2015020414.json
../datasets/raw/2015\regular_season\2015020415.json
../datasets/raw/2015\regular_season\2015020416.json
../datasets/raw/2015\regular_season\2015020417.json
../datasets/raw/2015\regular_season\2015020418.json
../datasets/raw/2015\regular_season\2015020419.json
../datasets/raw/2015\regular_season\2015020420.json
../datasets/raw/2015\regular_season\2015020421.json
../datasets/raw/2015\regular_season\2015020422.json
../datasets/raw/2015\regular_season\2015020423.json
../datasets/raw/2015\regular_season\2015020424.json
../datasets/raw/2015\regular_season\2015020425.json
../datasets/raw/2015\regular_season\2015020426.json
../datasets/raw/2015\regular_season\2015020427.json
../datasets/raw/2015\regular_season\2015020428.json
../datasets/

../datasets/raw/2015\regular_season\2015020580.json
../datasets/raw/2015\regular_season\2015020581.json
../datasets/raw/2015\regular_season\2015020582.json
../datasets/raw/2015\regular_season\2015020583.json
../datasets/raw/2015\regular_season\2015020584.json
../datasets/raw/2015\regular_season\2015020585.json
../datasets/raw/2015\regular_season\2015020586.json
../datasets/raw/2015\regular_season\2015020587.json
../datasets/raw/2015\regular_season\2015020588.json
../datasets/raw/2015\regular_season\2015020589.json
../datasets/raw/2015\regular_season\2015020590.json
../datasets/raw/2015\regular_season\2015020591.json
../datasets/raw/2015\regular_season\2015020592.json
../datasets/raw/2015\regular_season\2015020593.json
../datasets/raw/2015\regular_season\2015020594.json
../datasets/raw/2015\regular_season\2015020595.json
../datasets/raw/2015\regular_season\2015020596.json
../datasets/raw/2015\regular_season\2015020597.json
../datasets/raw/2015\regular_season\2015020598.json
../datasets/

...

../datasets/raw/2019\regular_season\2019020522.json
../datasets/raw/2019\regular_season\2019020523.json
../datasets/raw/2019\regular_season\2019020524.json
../datasets/raw/2019\regular_season\2019020525.json
../datasets/raw/2019\regular_season\2019020526.json
../datasets/raw/2019\regular_season\2019020527.json
../datasets/raw/2019\regular_season\2019020528.json
../datasets/raw/2019\regular_season\2019020529.json
../datasets/raw/2019\regular_season\2019020530.json
../datasets/raw/2019\regular_season\2019020531.json
../datasets/raw/2019\regular_season\2019020532.json
../datasets/raw/2019\regular_season\2019020533.json
../datasets/raw/2019\regular_season\2019020534.json
../datasets/raw/2019\regular_season\2019020535.json
../datasets/raw/2019\regular_season\2019020536.json
../datasets/raw/2019\regular_season\2019020537.json
../datasets/raw/2019\regular_season\2019020538.json
../datasets/raw/2019\regular_season\2019020539.json
../datasets/raw/2019\regular_season\2019020540.json
../datasets/

../datasets/raw/2019\regular_season\2019020681.json
../datasets/raw/2019\regular_season\2019020682.json
../datasets/raw/2019\regular_season\2019020683.json
../datasets/raw/2019\regular_season\2019020684.json
../datasets/raw/2019\regular_season\2019020685.json
../datasets/raw/2019\regular_season\2019020686.json
../datasets/raw/2019\regular_season\2019020687.json
../datasets/raw/2019\regular_season\2019020688.json
../datasets/raw/2019\regular_season\2019020689.json
../datasets/raw/2019\regular_season\2019020690.json
../datasets/raw/2019\regular_season\2019020691.json
../datasets/raw/2019\regular_season\2019020692.json
../datasets/raw/2019\regular_season\2019020693.json
../datasets/raw/2019\regular_season\2019020694.json
../datasets/raw/2019\regular_season\2019020695.json
../datasets/raw/2019\regular_season\2019020696.json
../datasets/raw/2019\regular_season\2019020697.json
../datasets/raw/2019\regular_season\2019020698.json
../datasets/raw/2019\regular_season\2019020699.json
../datasets/

../datasets/raw/2019\regular_season\2019020841.json
../datasets/raw/2019\regular_season\2019020842.json
../datasets/raw/2019\regular_season\2019020843.json
../datasets/raw/2019\regular_season\2019020844.json
../datasets/raw/2019\regular_season\2019020845.json
../datasets/raw/2019\regular_season\2019020846.json
../datasets/raw/2019\regular_season\2019020847.json
../datasets/raw/2019\regular_season\2019020848.json
../datasets/raw/2019\regular_season\2019020849.json
../datasets/raw/2019\regular_season\2019020850.json
../datasets/raw/2019\regular_season\2019020851.json
../datasets/raw/2019\regular_season\2019020852.json
../datasets/raw/2019\regular_season\2019020853.json
../datasets/raw/2019\regular_season\2019020854.json
../datasets/raw/2019\regular_season\2019020855.json
../datasets/raw/2019\regular_season\2019020856.json
../datasets/raw/2019\regular_season\2019020857.json
../datasets/raw/2019\regular_season\2019020858.json
../datasets/raw/2019\regular_season\2019020859.json
../datasets/

../datasets/raw/2019\regular_season\2019021001.json
../datasets/raw/2019\regular_season\2019021002.json
../datasets/raw/2019\regular_season\2019021003.json
../datasets/raw/2019\regular_season\2019021004.json
../datasets/raw/2019\regular_season\2019021005.json
../datasets/raw/2019\regular_season\2019021006.json
../datasets/raw/2019\regular_season\2019021007.json
../datasets/raw/2019\regular_season\2019021008.json
../datasets/raw/2019\regular_season\2019021009.json
../datasets/raw/2019\regular_season\2019021010.json
../datasets/raw/2019\regular_season\2019021011.json
../datasets/raw/2019\regular_season\2019021012.json
../datasets/raw/2019\regular_season\2019021013.json
../datasets/raw/2019\regular_season\2019021014.json
../datasets/raw/2019\regular_season\2019021015.json
../datasets/raw/2019\regular_season\2019021016.json
../datasets/raw/2019\regular_season\2019021017.json
../datasets/raw/2019\regular_season\2019021018.json
../datasets/raw/2019\regular_season\2019021019.json
../datasets/

../datasets/raw/2019\regular_season\2019021166.json
../datasets/raw/2019\regular_season\2019021167.json
../datasets/raw/2019\regular_season\2019021168.json
../datasets/raw/2019\regular_season\2019021169.json
../datasets/raw/2019\regular_season\2019021170.json
../datasets/raw/2019\regular_season\2019021171.json
../datasets/raw/2019\regular_season\2019021172.json
../datasets/raw/2019\regular_season\2019021173.json
../datasets/raw/2019\regular_season\2019021174.json
../datasets/raw/2019\regular_season\2019021175.json
../datasets/raw/2019\regular_season\2019021176.json
../datasets/raw/2019\regular_season\2019021177.json
../datasets/raw/2019\regular_season\2019021178.json
../datasets/raw/2019\regular_season\2019021179.json
../datasets/raw/2019\regular_season\2019021180.json
../datasets/raw/2019\regular_season\2019021181.json
../datasets/raw/2019\regular_season\2019021182.json
../datasets/raw/2019\regular_season\2019021183.json
../datasets/raw/2019\regular_season\2019021184.json
../datasets/

In [68]:
# transform into a dataframe
columns_name = [
"event_Idx","period", "periodTime","id_team_that_shot","name_team_that_shot","result_event","x_coord","y_coord","distance","angle","rinkSide_of_the_team_that_shot","goalie_name",\
        "shooter_name","shot_type","empty_net","strength","last_event_type","last_x_coord","last_y_coord","distance_from_last","seconds_since_last","rebound","angle_change","speed",\
        "type_season","id_game","season","dateTime","endDateTime","abstractGameState","team_away_name","team_home_name"]
df = pd.DataFrame(all_list_data, columns=columns_name)
pd.set_option('display.max_columns', None)
df.head(20)

,event_Idx,period,periodTime,id_team_that_shot,name_team_that_shot,result_event,x_coord,y_coord,distance,angle,rinkSide_of_the_team_that_shot,goalie_name,shooter_name,shot_type,empty_net,strength,last_event_type,last_x_coord,last_y_coord,distance_from_last,seconds_since_last,rebound,angle_change,speed,type_season,id_game,season,dateTime,endDateTime,abstractGameState,team_away_name,team_home_name
0,9,1,01:27,13,Florida Panthers,Shot,-35.0,31.0,63.134776,0.513252,right,Thomas Greiss,Alex Petrovic,Slap Shot,None,None,Faceoff,-20.0,-22.0,55.081757,12.0,False,0.000000,4.590146,playoffs,2015030111,20152016,2016-04-15T00:00:00Z,2016-04-15T02:49:01Z,Final,New York Islanders,Florida Panthers
1,12,1,01:55,13,Florida Panthers,Goal,-78.0,5.0,13.000000,0.394791,right,Thomas Greiss,Teddy Purcell,Tip-In,False,Even,Faceoff,-69.0,22.0,19.235384,27.0,False,0.000000,0.712422,playoffs,2015030111,20152016,2016-04-15T00:00:00Z,2016-04-15T02:49:01Z,Final,New York Islanders,Florida Panthers
2,14,1,02:06,13,Florida Panthers,Shot,-83.0,-20.0,21.189620,-1.234122,right,Thomas Greiss,Nick Bjugstad,Backhand,None,None,Faceoff,0.0,0.0,85.375641,11.0,False,0.000000,7.761422,playoffs,2015030111,20152016,2016-04-15T00:00:00Z,2016-04-15T02:49:01Z,Final,New York Islanders,Florida Panthers
3,18,1,02:54,2,New York Islanders,Shot,48.0,25.0,48.877398,0.536911,left,Roberto Luongo,Thomas Hickey,Wrist Shot,None,None,Blocked Shot,73.0,9.0,29.681644,2.0,False,0.000000,14.840822,playoffs,2015030111,20152016,2016-04-15T00:00:00Z,2016-04-15T02:49:01Z,Final,New York Islanders,Florida Panthers
4,19,1,02:59,13,Florida Panthers,Shot,-57.0,14.0,35.846897,0.401229,right,Thomas Greiss,Aleksander Barkov,Snap Shot,None,None,Shot,48.0,25.0,105.574618,5.0,True,0.222013,21.114924,playoffs,2015030111,20152016,2016-04-15T00:00:00Z,2016-04-15T02:49:01Z,Final,New York Islanders,Florida Panthers
5,26,1,04:39,13,Florida Panthers,Shot,-49.0,-17.0,44.384682,-0.393058,right,Thomas Greiss,Brian Campbell,Slap Shot,None,None,Giveaway,-39.0,-38.0,23.259407,0.5,False,0.000000,46.518813,playoffs,2015030111,20152016,2016-04-15T00:00:00Z,2016-04-15T02:49:01Z,Final,New York Islanders,Florida Panthers
6,35,1,06:26,2,New York Islanders,Shot,84.0,10.0,11.661904,1.030377,left,Roberto Luongo,Shane Prince,Wrap-around,None,None,Hit,95.0,-37.0,48.270074,29.0,False,0.000000,1.664485,playoffs,2015030111,20152016,2016-04-15T00:00:00Z,2016-04-15T02:49:01Z,Final,New York Islanders,Florida Panthers
7,37,1,06:39,2,New York Islanders,Goal,63.0,1.0,27.018512,0.037020,left,Roberto Luongo,Brock Nelson,Snap Shot,False,Even,Hit,-7.0,-39.0,80.622577,4.0,False,0.000000,20.155644,playoffs,2015030111,20152016,2016-04-15T00:00:00Z,2016-04-15T02:49:01Z,Final,New York Islanders,Florida Panthers
8,40,1,07:10,2,New York Islanders,Shot,75.0,13.0,19.849433,0.714091,left,Roberto Luongo,Brock Nelson,Wrist Shot,None,None,Giveaway,31.0,-11.0,50.119856,5.0,False,0.000000,10.023971,playoffs,2015030111,20152016,2016-04-15T00:00:00Z,2016-04-15T02:49:01Z,Final,New York Islanders,Florida Panthers
9,48,1,09:19,13,Florida Panthers,Shot,-61.0,12.0,31.384710,0.392340,right,Thomas Greiss,Teddy Purcell,Wrist Shot,None,None,Hit,85.0,-40.0,154.983870,23.0,False,0.000000,6.738429,playoffs,2015030111,20152016,2016-04-15T00:00:00Z,2016-04-15T02:49:01Z,Final,New York Islanders,Florida Panthers


In [69]:
df.to_csv("../datasets/tidy_data_Part_2_Q4.csv", index=False)